## PAVICS Web Processing Services using OGC-API integration with Weaver

When [Weaver component](https://github.com/bird-house/birdhouse-deploy/tree/master/birdhouse/components#weaver)
is enabled, all WPS *birds* registered as process *providers* will be automatically accessible using
[OGC-API - Processes](https://github.com/opengeospatial/ogcapi-processes/) interface from the endpoint where
[Weaver](https://github.com/crim-ca/weaver) is defined.

**NOTE**:
This test will only work when the component is enabled, and therefore requires explicit specification of test execution
using `WEAVER_TEST_ENABLED = true`


In [ ]:
import requests
import os
import sys

WEAVER_TEST_ENABLED=str(os.getenv("WEAVER_TEST_ENABLED", "false")).lower()
if WEAVER_TEST_ENABLED not in ["true", "yes", "1"]:
    print("Weaver test not enabled. Skipping...")
    sys.exit(0)


WEAVER_TEST_HOST=os.getenv("WEAVER_TEST_HOST", "pavics.ouranos.ca")
WEAVER_TEST_URL=os.getenv("WEAVER_TEST_URL", "https://{}/weaver".format(WEAVER_TEST_HOST))
WEAVER_TEST_DEFAULT_BIRDS = "catalog, finch, flyingpigeon, hummingbird, malleefowl, raven"
WEAVER_TEST_KNOWN_BIRDS=os.getenv("WEAVER_TEST_KNOWN_BIRDS", WEAVER_TEST_DEFAULT_BIRDS)
WEAVER_TEST_KNOWN_BIRDS=list(bird.strip() for bird in WEAVER_TEST_KNOWN_BIRDS.split(","))
assert len(WEAVER_TEST_KNOWN_BIRDS) >= 1, "No test WPS provider provided in 'WEAVER_TEST_KNOWN_BIRDS'."

WEAVER_TEST_DEFAULT_FILE = "/twitcher/ows/proxy/thredds/dodsC/birdhouse/nrcan/nrcan_canada_daily/tasmin/nrcan_canada_daily_tasmin_2013.nc"
WEAVER_TEST_FILE = os.getenv("WEAVER_TEST_FILE", "https://{}{}".format(WEAVER_TEST_HOST, WEAVER_TEST_DEFAULT_FILE))

WEAVER_HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

### Start with simple listing of registered WPS providers in Weaver


In [ ]:
path = "{}/providers".format(WEAVER_TEST_URL)
resp = requests.get(path, headers=WEAVER_HEADERS)
assert resp.status_code == 200, "Error during WPS bird providers listing:\n[{}]".format(resp.text)
body = resp.json()
print(body)

assert "providers" in body and len(body["providers"]), "Could not find Weaver WPS providers"
bird_ids = [bird["id"] for bird in body["providers"]]
assert all(bird in bird_ids for bird in WEAVER_TEST_KNOWN_BIRDS), "Could not find all expected Weaver WPS providers"


### Obtain OGC-API converted WPS processes by Weaver from original WPS providers endpoints

For each registered provider, Weaver sends a *GetCapabilities* WPS request to the remote endpoint and parses the XML
result in order to form the corresponding OGC-API JSON content.

In [ ]:
print("All WPS provider processes converted to OGC-API interface by Weaver:\n")
all_processes = []
for bird in bird_ids:
    path = "{}/providers/{}/processes".format(WEAVER_TEST_URL, bird)
    resp = requests.get(path, headers=WEAVER_HEADERS)
    assert resp.status_code == 200, "Error during WPS bird processes retrieval:\n[{}]".format(resp.text)
    body = resp.json()
    for process in body["processes"]:
        process_desc_url = "{}/{}".format(path, process["id"])
        all_processes.append(process_desc_url)
        print(" -", process_desc_url)
assert len(all_processes), "Could not find any process!"

### Dispatched execution of Flyingpigeon WPS process

Here, we attempt running the same process defined in [WPS_example Notebook](WPS_example.ipynb), but through the OGC-API
interface provided by Weaver.

The process execution received by Weaver gets dispatched to the real WPS location. Weaver then monitors the process
until completion and, once completed, returns the location where results can be retrieved.

In [ ]:
if not "hummingbird" in WEAVER_TEST_KNOWN_BIRDS:
    print("Hummingbird not specified within known WPS provider birds by Weaver. Skipping dispatched execution test...")
    sys.exit(0)

WEAVER_BIRD_URL = "{}/providers/hummingbird".format(WEAVER_TEST_URL)
WEAVER_BIRD_PROCESS = "{}/processes/ncdump".format(WEAVER_BIRD_URL)
assert WEAVER_BIRD_PROCESS in all_processes, "Could not find WPS bird process URL to test execution."


#### First let's obtain the specific description of the test WPS process

This request will tell us the explicit details of the process such as its inputs, outputs, and other metadata.
Once again, Weaver parses the results retrieved from the original WPS provider (using *DescribeProcess* this time),
to generate the corresponding outputs. Weaver also adds additional metadata when it can infer some missing details
from returned description fields.

In [ ]:
resp = requests.get(WEAVER_BIRD_PROCESS, headers=WEAVER_HEADERS)
assert resp.status_code == 200, "Error getting WPS process description:\n[{}]".format(resp.text)
body = resp.json()
print(body)

#### Submit the new process execution

Using OGC-API interface, WPS process execution are accomplished using a *Job*. That job will tell us the status
location where we can monitor the process execution.

From the previous response, we can see that the process accepts many inputs and format variations.
In this case, we are interested in the input named `dataset` to submit the file defined by `WEAVER_TEST_FILE`.

Following execution of the process, we expect to obtain a raw text data dump of the test file content. 
The location of the raw text file is expected be provided by output named `output` according to the process description.

In [ ]:
print("Testing with:")
print("  File:     [{}]".format(WEAVER_TEST_FILE))
print("  Process:  [{}]".format(WEAVER_BIRD_PROCESS))

data = {
  "mode": "async",  # This tells Weaver to run the process asynchronously, such that we get non-blocking status location
  "response": "document",  # Type of status response (only this mode supported for the time being)
  "inputs": [
    {
      "id": "dataset",  # The target input
      "href": WEAVER_TEST_FILE
    }
  ],
  "outputs": [
    {
      "id": "output",   # Target output we want to retrieve
      "transmissionMode": "reference"  # Ask to provide the result as HTTP reference 
    }
  ]
}

